In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import networkx as nx
%matplotlib inline
# from DYNASMART_Process import *
from Modules import DYNASMART_Process as dy
# import preprocessing_read_skim_file as rs
from Modules import AV_functions as av
from Modules import Solve_DARP as sod
from Modules import Postprocess_DARP as pod
from Modules import Preprocess_DARP as prd
from Modules import DARP_to_DYNA as dtd
from gurobipy import *
from Modules import *
import datetime
import requests
import math
import importlib
import random
%load_ext autoreload
%autoreload 2

In [2]:
#Read in the basic traveler and trips informationa and joint the houshold id
# traveler_info=pd.read_csv('input/traveler.csv')
personData=pd.read_csv('input/personData_1.csv')
person_type_dic=dict(zip(personData.person_id,personData.type))
traveler_trips=pd.read_csv('input/traveler_trip_info_improved.csv',index_col=0)
num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id=dy.read_network('input/network.dat')
nodexy=dy.read_xy('input/xy.dat')
superzone_filepath='Input/SuperZone.dat'
linktraveltime_filepath='Input/output_td_linktraveltime.dat'
superzone_map=dy.read_superzone_info(superzone_filepath)
origin_path='Input/origin.dat'
destination_path='Input/destination.dat'
origins,destinations,origin_links=dy.read_origin_destion(origin_path,destination_path)
origin_links['length']=origin_links.apply(lambda row: 
                                          link_detail[int(link_id[node_id[row.u_node],node_id[row.d_node]])][0,4],axis=1)
VPNRSkim_folder_filepath='Input/vehicularPNRSkim'
Vehicular_Skim,Vehicular_Skim_Dict=dy.read_vehicle_skim(VPNRSkim_folder_filepath)
transit_skim_folderpath='Input/'
TransitMazTazFlag,drivingweight,walkingweight,\
TransitSkimTimeIntervalLength,MaxNumTransitSkimTimeInterval,WalkSpeed,transit_zone_candidates,\
Transit_AB_Cost_Skim,Transit_AB_Time_Skim,three_link_walk=dy.read_transitskim(transit_skim_folderpath)
Transit_AB_Cost_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Cost_Skim,TransitSkimTimeIntervalLength)
Transit_AB_Time_Skim_Dict=dy.convert_skim_dataframe_to_dict(Transit_AB_Time_Skim,TransitSkimTimeIntervalLength)
transit_zone_dict={}
for link_zone in three_link_walk.three_link_zone.unique():
    transit_zone_dict[link_zone]=list(three_link_walk[three_link_walk.three_link_zone==link_zone]['transit_zone'].astype(int))
three_link_walk_dict={}
for index,group in three_link_walk.groupby(['three_link_zone']):
    three_link_walk_dict[index]={}
    for inx,row in group.iterrows():
        three_link_walk_dict[index][row.transit_zone]=row.distance
intrasuperzone_vehicle_filepath='Input/intrasuperzone_vehicle.dat'
intrasuperzone_path_filepath='Input/intrasuperzone_path.dat'
drivingcost_per_mile=0.5
intrasuperzone_info,intrasuperzone_path_dic=dtd.read_intrasuperzone_files(intrasuperzone_vehicle_filepath,intrasuperzone_path_filepath)
traveler_trips=prd.compare_mode_utlity(traveler_trips,TransitMazTazFlag,
                        three_link_walk_dict,Transit_AB_Cost_Skim_Dict,
                        Transit_AB_Time_Skim_Dict,TransitSkimTimeIntervalLength,
                        Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile,transit_zone_dict)

56 57 72


In [3]:
car_trips=traveler_trips.loc[(traveler_trips.tripmode<=6) &(traveler_trips.driver_passenger_flag>=1)]
car_trips=car_trips.loc[(car_trips.orig_taz !=car_trips.dest_taz) | (car_trips.trip_counter==1)]
car_transit_trips=car_trips.append(traveler_trips[traveler_trips.tripmode>8])
# car_transit_trips=prd.compare_mode_utlity(car_transit_trips,TransitMazTazFlag,
#                         three_link_walk_dict,Transit_AB_Cost_Skim_Dict,
#                         Transit_AB_Time_Skim_Dict,TransitSkimTimeIntervalLength,
#                         Vehicular_Skim_Dict,superzone_map,drivingcost_per_mile,transit_zone_dict)
random.seed(100)
car_transit_trips_sample=car_transit_trips[car_transit_trips.hh_id.isin(random.sample(list(car_transit_trips.hh_id.unique()),3000))]
car_trips_sample=car_trips[car_trips.hh_id.isin(random.sample(list(car_transit_trips.hh_id.unique()),3000))]

In [4]:
car_trips_shopping_special=car_trips.copy()
car_trips_shopping_special.loc[((car_trips_shopping_special.orig_purpose=='Shop') \
                           |(car_trips_shopping_special.dest_purpose=='Shop')),\
                               ['earlyarr_penalty', 'latearr_penalty','early_penalty_threshold', 'late_penalty_threshold']]\
                            =0

In [709]:
deviation_list=[]
num_trips=[]
hh_id_list=[]
target_darp_solutions=darp_solutions_modechoice05vot
for hh_id in car_transit_trips.hh_id.unique():
    hh_id_list.extend([hh_id])
    num_trips.extend([target_darp_solutions[hh_id]['num_pickup_trips']])
    deviation_list.extend([target_darp_solutions[hh_id]['schedule_deviation']])
deviation=pd.DataFrame({'hh_id':hh_id_list,'num_trips':num_trips,'deviation_list':deviation_list})
deviation.deviation_list.sum()/deviation.num_trips.sum()

1.1787082975171714

In [593]:
output_flag=1
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*5
drivingcost_per_mile=0.5
reward_mode=0
run_mode=2
darp_solutions=[]
cav_use_mode=0
num_time_interval=5
if num_time_interval==5:
    TL=[0,270,420,780,960]
    TU=[270,420,780,960,1440]
else:
    TL=[i*1440/num_time_interval for i in range(num_time_interval)]
    TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0

print('********************************')
print(share_ride_factor)
# target_trips=car_trips_sample[car_trips_sample.hh_id==1486727]

route_info_modechoice_sample,darp_solutions_modechoice_sample=\
sod.get_route_info_allhh(car_trips_sample[car_trips_sample.hh_id==2104157],
                         output_flag,min_length,max_length,single_model_runtime,drivingcost_per_mile,
                         reward_mode,run_mode,cav_use_mode,num_time_interval,num_cav,share_ride_factor
                         ,time_window_flag,Vehicular_Skim_Dict,
                         Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                         superzone_map,TL,TU,
                         transit_zone_dict,transit_zone_candidates,TransitMazTazFlag,
                         TransitSkimTimeIntervalLength )

********************************
1.5
Estimate Route for the  0 th household  2018-08-06 15:24:08.479817
2018-08-06 15:24:08.483817 	 18 	 2104157
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TIME_LIMIT to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 9195 rows, 7657 columns and 301471 nonzeros
Variable types: 76 continuous, 7581 integer (7581 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+03]
  Objective range  [1e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]

MIP start did not produce a new incumbent solution
MIP start violates constraint precedencet15[18,36,0,4] by 0.700000000

Found heuristic solution: objective -316977.6800
Presolve removed 4059 rows and 4603 columns
Presolve time: 1.28s
Presolved: 5136 rows, 3054 columns, 49695 nonzeros
Variable types: 76 continuous, 2978 integer (2978 binary)

Root relaxation: objective 1.039411e+03, 1071 ite

     0     0 1021.43662    0  170  633.49027 1021.43662  61.2%     -   16s
     0     0 1021.36094    0  163  633.49027 1021.36094  61.2%     -   16s
     0     0 1021.34648    0  167  633.49027 1021.34648  61.2%     -   17s
     0     0 1021.34187    0  166  633.49027 1021.34187  61.2%     -   17s
     0     0 1021.30414    0  153  633.49027 1021.30414  61.2%     -   17s
H    0     0                     679.0885100 1021.30414  50.4%     -   17s
     0     0 1021.28035    0  163  679.08851 1021.28035  50.4%     -   17s
     0     0 1021.27067    0  189  679.08851 1021.27067  50.4%     -   17s
     0     0 1021.26776    0  171  679.08851 1021.26776  50.4%     -   17s
     0     0 1021.24279    0  135  679.08851 1021.24279  50.4%     -   18s
     0     0 1021.23967    0  146  679.08851 1021.23967  50.4%     -   18s
H    0     0                     719.9000500 1021.23967  41.9%     -   18s
     0     0 1021.22908    0  158  719.90005 1021.22908  41.9%     -   18s
     0     0 1021.22565  

In [8]:
reward_mode=0
target_hh_id=2104157
output_flag=1
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*5
drivingcost_per_mile=0.5
reward_mode=0
run_mode=2
darp_solutions=[]
cav_use_mode=0
num_time_interval=1
if num_time_interval==5:
    TL=[0,270,420,780,960]
    TU=[270,420,780,960,1440]
else:
    TL=[i*1440/num_time_interval for i in range(num_time_interval)]
    TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1.5
time_window_flag=0
target_hh=traveler_trips[traveler_trips['hh_id']==target_hh_id].drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose',
    'dest_purpose','starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
#hh_index give an index to all trips within the household for tracking purpose
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,\
early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone\
    =prd.extract_hh_information(sorted_trips,Vehicular_Skim_Dict,\
                                Transit_AB_Cost_Skim_Dict,superzone_map,drivingcost_per_mile,num_time_interval)
R=prd.estimate_trip_reward(hh_num_trips,sorted_trips,Vehicular_Skim_Dict,Transit_AB_Cost_Skim_Dict,
                           Transit_AB_Time_Skim_Dict,three_link_walk_dict,reward_mode,superzone_map
                           ,drivingcost_per_mile,transit_zone_dict,transit_zone_candidates,TransitMazTazFlag,
                           TransitSkimTimeIntervalLength)
num_cav=1
run_mode=0
num_time_interval=1
single_model_runtime=60*30
m1,x,T,obj1,obj2,obj3=sod.dial_n_ride_model(num_hh_member,hh_num_trips,C,TT,sorted_trips,
                expected_arrival_time,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,
                R,Vehicular_Skim_Dict,share_ride_factor,output_flag,run_mode,reward_mode,num_cav,cav_use_mode,time_window_flag,single_model_runtime)
route_info=sod.extract_route_from_model_solution(x,T,R,TT,sorted_trips,visit_candidate_zone,hh_num_trips,expected_arrival_time,expected_leave_time,superzone_map,num_cav,num_time_interval,run_mode)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TIME_LIMIT to 1800.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 6676 rows, 5248 columns and 49840 nonzeros
Variable types: 128 continuous, 5120 integer (5120 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+03]
  Objective range  [1e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]

MIP start did not produce a new incumbent solution
MIP start violates constraint precedencet15[62,63,0] by 0.610000000

Found heuristic solution: objective -313260.1476
Presolve removed 2022 rows and 2925 columns
Presolve time: 0.38s
Presolved: 4654 rows, 2323 columns, 27796 nonzeros
Variable types: 128 continuous, 2195 integer (2195 binary)

Root relaxation: objective 1.860788e+03, 1083 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [9]:
num_time_interval=5
if num_time_interval==5:
    TL=[0,270,420,780,960]
    TU=[270,420,780,960,1440]
else:
    TL=[i*1440/num_time_interval for i in range(num_time_interval)]
    TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_hh_member,hh_num_trips,C,TT,expected_arrival_time,expected_leave_time,\
early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,visit_candidate_zone\
    =prd.extract_hh_information(sorted_trips,Vehicular_Skim_Dict,\
                                Transit_AB_Cost_Skim_Dict,superzone_map,drivingcost_per_mile,num_time_interval)

In [16]:
x_sol=np.zeros([2*hh_num_trips+2,2*hh_num_trips+2,num_cav])
T_sol=np.zeros([2*hh_num_trips+2])
for i in range(2*hh_num_trips+2):
    for j in range(2*hh_num_trips+2):
        for ve in range(num_cav):
            x_sol[i,j,ve]=x[i,j,ve].x
    T_sol[i]=T[i].x

In [93]:
for i in range(2*hh_num_trips+2):
    print(T_sol[i]==round(T[i].x,2),T_sol[i],round(T[i].x,2))

True 242.0 242.0
True 257.61 257.61
True 242.01 242.01
False 265.31 265.31
True 365.0 365.0
True 383.0 383.0
True 391.0 391.0
False 410.0 410.0
True 429.0 429.0
False 443.0 443.0
True 461.0 461.0
True 495.0 495.0
True 560.0 560.0
True 587.0 587.0
True 683.0 683.0
False 751.7 751.6
False 759.8 759.8
False 755.6 755.5
False 774.15 774.8
False 799.55 800.2
False 788.25 788.9
False 820.0 820.0
True 890.0 890.0
False 914.0 896.1
True 1047.0 1047.0
True 1106.0 1106.0
True 1118.7 1118.7
False 1127.9 1123.4
False 1159.4 1159.4
True 1167.0 1167.0
True 1319.0 1319.0
False 1431.39 1431.29
False 262.21 262.21
False 246.61 246.61
False 268.41 268.41
True 367.0 367.0
True 384.9 384.9
True 400.5 400.5
True 418.9 418.9
False 437.9 437.9
False 452.3 452.3
True 462.7 462.7
True 504.2 504.2
True 569.6 569.6
True 594.3 594.3
True 690.0 690.0
True 757.1 757.1
False 763.0 763.0
False 772.55 773.2
False 783.25 783.9
False 808.65 809.3
False 797.95 798.6
True 826.6 826.6
True 896.1 896.1
True 917.0 917.0
True

In [89]:
m3,T=sod.optimal_start_time(sorted_trips,x_sol,T_sol,C,TT,TL,TU,num_time_interval,num_cav,hh_num_trips,expected_arrival_time,Vehicular_Skim_Dict,
    visit_candidate_zone,early_penalty,late_penalty,early_penalty_threshold,late_penalty_threshold,superzone_map)

[    0.   142.   242.   342.  1682.]
alexcheck 100.0 100.0
{0: 2, 1: 32, 2: 33, 3: 34, 4: 35, 5: 36, 6: 37, 7: 38, 8: 39, 9: 40, 10: 41, 11: 42, 12: 43, 13: 44, 14: 45, 15: 17, 16: 47, 17: 46, 18: 49, 19: 50, 20: 51, 21: 52, 22: 53, 23: 54, 24: 55, 25: 56, 26: 27, 27: 57, 28: 59, 29: 60, 30: 61, 31: 62, 32: 3, 33: 1, 34: 4, 35: 5, 36: 6, 37: 7, 38: 8, 39: 9, 40: 10, 41: 11, 42: 12, 43: 13, 44: 14, 45: 15, 46: 16, 47: 48, 48: 18, 49: 20, 50: 21, 51: 19, 52: 22, 53: 23, 54: 24, 55: 25, 56: 26, 57: 58, 58: 28, 59: 29, 60: 30, 61: 31, 62: 63}
Optimize a model with 1084 rows, 448 columns and 2550 nonzeros
Model has 127 piecewise-linear objective terms
Variable types: 128 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+03]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 130 rows and 8 columns
Presolve time: 0.04s
Presolved: 1391 rows, 1090 columns, 3590 nonzeros
Variable types: 6

In [303]:
output_flag=0
# target_hh_id=2744264
min_length=15
max_length=20
single_model_runtime=60*3
drivingcost_per_mile=0.5
reward_mode=0
run_mode=2
darp_solutions=[]
cav_use_mode=0
num_time_interval=2
TL=[i*1440/num_time_interval for i in range(num_time_interval)]
TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
num_cav=1
share_ride_factor=1
time_window_flag=0

In [489]:
target_hh=car_trips_shopping_special[(car_trips_shopping_special['hh_id']==target_hh_id)]\
.drop_duplicates(subset=['orig_maz','dest_maz','orig_purpose','dest_purpose',
                         'starttime','joint_trip_flag'])
#Sort all trips based on start time. This step could reduce the solving time and make it easier to track
sorted_trips=target_hh.sort_values("starttime")
hh_num_trips=sorted_trips.shape[0]
sorted_trips["hh_index"]=(range(hh_num_trips))
darp_solutions={}
run_mode=2
num_time_interval=5
if num_time_interval==5:
    TL=[0,270,420,780,960]
    TU=[270,420,780,960,1440]
else:
    TL=[i*1440/num_time_interval for i in range(num_time_interval)]
    TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
darp_solutions_schedule_adjustement={}
darp_solutions_schedule_adjustement[target_hh_id]=sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim_Dict,Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                                    superzone_map,min_length,max_length,reward_mode,drivingcost_per_mile,share_ride_factor,output_flag,run_mode,num_cav,
                                    cav_use_mode,time_window_flag,single_model_runtime,num_time_interval,TL,TU,transit_zone_dict,transit_zone_candidates,
                                    TransitMazTazFlag,TransitSkimTimeIntervalLength)

In [469]:
run_mode=0
num_time_interval=1
reward_mode=1
if num_time_interval==5:
    TL=[0,270,420,780,960]
    TU=[270,420,780,960,1440]
else:
    TL=[i*1440/num_time_interval for i in range(num_time_interval)]
    TU=[(i+1)*1440/num_time_interval for i in range(num_time_interval)]
# sorted_trips=sorted_trips.loc[sorted_trips.tripmode<=6]
darp_solutions={}
darp_solutions[target_hh_id]=sod.solve_with_schedule_partition(sorted_trips,Vehicular_Skim_Dict,Transit_AB_Cost_Skim_Dict,Transit_AB_Time_Skim_Dict,three_link_walk_dict,
                                    superzone_map,min_length,max_length,reward_mode,drivingcost_per_mile,share_ride_factor,output_flag,run_mode,num_cav,
                                    cav_use_mode,time_window_flag,single_model_runtime,num_time_interval,TL,TU,transit_zone_dict,transit_zone_candidates,
                                    TransitMazTazFlag,TransitSkimTimeIntervalLength)